(This notebook is for **the week 3** assignment of the **Coursera Applied Data Science Capstone** module.)
### Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# install web-scraper (following: https://youtu.be/ng2o98k983k)
#! pip install beautifulsoup4 # uncomment if not installed
#! pip install lxml # uncomment if not installed
#! pip intsall html5lib # uncomment if not installed
#! pip install requests # uncomment if not installed

Import libraries and get wikipedia page content.

In [2]:
import pandas as pd # import pandas library
import requests # import requests library
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # url to be scraped
page_response = requests.get(url) # fetch the url content using the requests library
src=page_response.content # set the content of the repsonse to a variable #https://realpython.com/python-requests/

Import the beautiful soup library, create the soup object and output the page title.

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(src, 'lxml') # pass the variable to BeautifulSoup, https://youtu.be/87Gx3U0BDlo                    
#print(soup)
soup.title.text # output the title of the soup object #https://www.pluralsight.com/guides/extracting-data-html-beautifulsoup

'List of postal codes of Canada: M - Wikipedia'

Create a variable for the table only. Find all the tr tagged data in the table.

In [4]:
table=soup.find("table",{"class":"wikitable sortable"})  #https://stackoverflow.com/questions/33766740/beautifulsoup-find-table-with-specified-class-on-wikipedia-page
table_data = table.tbody.find_all("tr") # in the tabe body find all tr tags 
# uncomment a line to check the data
#print(table) 
#df.head() 
#print(table.title) 
#print(table_data) 

Get and print the table headings.

In [5]:
headings = []
for th in table_data[0].find_all("th"):
    headings.append(th.text.replace('\n',''))
headings[2]='Neighborhood'
print(headings) # list of table headings

['Postcode', 'Borough', 'Neighborhood']


Get the data from the table for one row to check it.

In [6]:
data_list = []
for td in table_data[287].find_all("td"):
    data_list.append(td.text.replace('\n',''))
print(data_list)

['M9Z', 'Not assigned', 'Not assigned']


Get the data from the table for **all rows**.

In [7]:
data_list = [] # variable ofr list off all data we want
for i in range(0,len(table_data)-1):
    for td in table_data[i].find_all("td"): # finds all the td tags in the html
        data_list.append(td.text.replace('\n','')) # add the row to the list
#print(data_list) # uncomment to check

Sanity check the result.

In [8]:
print('Length of list = ', len(data_list), sep='')
print('Number of columns = ', len(headings), sep='')
print('Number of rows in the table = ', int(len(data_list)/len(headings)), sep='')

Length of list = 858
Number of columns = 3
Number of rows in the table = 286


Reshape the list as an array and cast to a dataframe.

In [9]:
import numpy as np
reshaped_array=np.reshape(np.asarray(data_list), (int(len(data_list)/len(headings)) , len(headings))) # reshape the array
df=pd.DataFrame(reshaped_array) # make the result a dataframe
df.columns=headings # assign the table headings as the column names
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Condition to find the 'Not Assigned' **Boroughs**, and remove them.

In [10]:
condition=df['Borough']=='Not assigned' # the condition that we don't want https://github.com/pandas-dev/pandas/issues/20944
df=df[~condition] # pick rows that do not (~) satisfy the condition
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Group the dataframe by **'Postcode'** and **'Borough'** and apply the join function to the grouped **'Neighborhood'** data.

In [11]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join) #https://realpython.com/pandas-groupby/
df=df.to_frame().reset_index() # reset index
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Condition to find the 'Not Assigned' **Neighborhoods**, and remove them.

In [12]:
condition=df['Neighborhood']=='Not assigned' # the condition that we don't want https://github.com/pandas-dev/pandas/issues/20944
for i in range(0,len(condition-1)):
    if condition[i]==True:
        df['Neighborhood'][i]=df['Borough'][i]      
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
print('Number of rows in the dataframe = ', df.shape[0], sep='')

Number of rows in the dataframe = 103


In [14]:
postal_code=df.loc[0]['Postcode'] # first bracket is the index of the row, second bracket is the column
postal_code

'M1B'

Instantiate new columns for latitude and longitude

In [15]:
df['Latitude']=None
df['Longitude']=None
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",None,None
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",None,None
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",None,None
3,M1G,Scarborough,Woburn,None,None
4,M1H,Scarborough,Cedarbrae,None,None


Use OpenCage Geocoding API (https://opencagedata.com/api) to get latitude and longitude

In [16]:
#!pip install opencage
from opencage.geocoder import OpenCageGeocode
key = 'ffa8658ade93408d9d4b84e1becce415'
geocoder = OpenCageGeocode(key)

for i in range(len(df)):
    postal_code=df.loc[i]['Borough'] + ', ' + df.loc[i]['Postcode']
    address = u'{}, Canada'.format(postal_code) # address to query
    results = geocoder.geocode(address)
    longitude = results[0]['geometry']['lng']
    latitude = results[0]['geometry']['lat']
    #print(address, latitude, longitude) # uncomment to check each result
    df.loc[i]['Latitude']=latitude # assign latitude in dataframe
    df.loc[i]['Longitude']=longitude # assign longitude in dataframe

In [17]:
df[0:12]

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.7731,-79.2578
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7731,-79.2578
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7627,-79.2309
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
5,M1J,Scarborough,Scarborough Village,43.7464,-79.2323
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7298,-79.2639
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7122,-79.2843
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7247,-79.2312
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6952,-79.2646


Create condition to filter the dataframe to Boroughs with Toronto in their name.

In [18]:
condition=df['Borough'].str.find('Toronto')!=-1
df_toronto=df[condition].reset_index()
df_toronto

,index,Postcode,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.6784,-79.2941
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.6693,-79.3155
3,43,M4M,East Toronto,Studio District,43.6561,-79.3406
4,44,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935
5,45,M4P,Central Toronto,Davisville North,43.7135,-79.3887
6,46,M4R,Central Toronto,North Toronto West,43.7143,-79.4065
7,47,M4S,Central Toronto,Davisville,43.702,-79.3853
8,48,M4T,Central Toronto,"Moore Park, Summerhill East",43.6899,-79.3853
9,49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6861,-79.4025


Create a map of Toronto with Neighborhoods superimposed on top.

In [19]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# get longitude and latitude of Toronto
results = geocoder.geocode('Toronto')
longitude = results[0]['geometry']['lng']
latitude = results[0]['geometry']['lat']
print(longitude,latitude)

-79.387207 43.653963


In [20]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = 'EIFEZ4ZQWKUH2DGAR45JXJB4RGZ34X5FQM3O5O2041VCP4Y5' # Foursquare ID
CLIENT_SECRET = '4QI3EVAC5L3REJPYSUD2BRSKOGZYNBTUJVO4121XMVLK5XVL' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: EIFEZ4ZQWKUH2DGAR45JXJB4RGZ34X5FQM3O5O2041VCP4Y5
CLIENT_SECRET:4QI3EVAC5L3REJPYSUD2BRSKOGZYNBTUJVO4121XMVLK5XVL


In [22]:
df_toronto.loc[0, 'Neighborhood']

'The Beaches'

Get the neighbouthood's latitude and longitude

In [23]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {} and {}, respectively.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.6784 and -79.2941, respectively.


**Get the top 100 venues that are in Marble Hill within a radius of 500 meters.**

First, create the GET request URL.

In [24]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EIFEZ4ZQWKUH2DGAR45JXJB4RGZ34X5FQM3O5O2041VCP4Y5&client_secret=4QI3EVAC5L3REJPYSUD2BRSKOGZYNBTUJVO4121XMVLK5XVL&v=20180605&ll=43.6784,-79.2941&radius=500&limit=100'

Send the GET request and examine the results.

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e245d98211536001c76f56e'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.682900004500006,
    'lng': -79.28788950308227},
   'sw': {'lat': 43.6738999955, 'lng': -79.30031049691773}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distance': 

The items key hast the information we want. Define a function that extracts the category from the items key.

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure into a Pandas dataframe.

In [27]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
#venues[0]
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bd461bc77b29c74a07d9282-0,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",4bd461bc77b29c74a07d9282,Glen Manor,CA,Toronto,Canada,Queen St.,...,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67682094413784...",43.676821,-79.293942,NaN,ON,Glen Manor Ravine,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad4c062f964a52011f820e3-1,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",4ad4c062f964a52011f820e3,125 Southwood Dr,CA,Toronto,Canada,NaN,...,"[125 Southwood Dr, Toronto ON M4E 0B8, Canada]","[{'label': 'display', 'lat': 43.678879, 'lng':...",43.678879,-79.297734,M4E 0B8,ON,The Big Carrot Natural Food Market,0,[],75150878
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c0e40c0c700c9b6e185a3dd-2,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",4c0e40c0c700c9b6e185a3dd,900 Kingston Rd.,CA,Toronto,Canada,Pickering Ave.,...,"[900 Kingston Rd. (Pickering Ave.), Toronto ON...","[{'label': 'display', 'lat': 43.68036312619922...",43.680363,-79.289692,NaN,ON,Beaches Bake Shop,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5286b7dd498e8b747c1dfe71-3,"[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",5286b7dd498e8b747c1dfe71,924 Kingston Rd.,CA,Toronto,Canada,NaN,...,"[924 Kingston Rd., Toronto ON, Canada]","[{'label': 'display', 'lat': 43.680493, 'lng':...",43.680493,-79.288846,NaN,ON,The Beech Tree,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b8daea1f964a520480833e3-4,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",4b8daea1f964a520480833e3,676 Kingston Rd.,CA,Toronto,Canada,at Main St.,...,"[676 Kingston Rd. (at Main St.), Toronto ON M4...","[{'label': 'display', 'lat': 43.67918143494101...",43.679181,-79.297215,M4E 1R4,ON,Grover Pub and Grub,0,[],NaN
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-56afcad6498e05333bf42031-5,"[{'id': '4bf58dd8d48988d162941735', 'name': 'O...",56afcad6498e05333bf42031,NaN,CA,NaN,Canada,NaN,...,[Canada],"[{'label': 'display', 'lat': 43.67629984029563...",43.676300,-79.294784,NaN,NaN,Glen Stewart Ravine,0,[],NaN
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4df91c4bae60f95f82229ad5-6,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",4df91c4bae60f95f82229ad5,NaN,CA,Toronto,Canada,NaN,...,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.68056321147582...",43.680563,-79.292869,NaN,ON,Upper Beaches,0,[],NaN
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4dbc8fe96a23e294ba3237bd-7,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4dbc8fe96a23e294ba3237bd,131 Glen Manor Drive,CA,Toronto,Canada,NaN,...,"[131 Glen Manor Drive, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67527822698259...",43.675278,-79.294647,NaN,ON,Glen Stewart Park,0,[],NaN


In [28]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Glen Manor Ravine,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",43.676821,-79.293942
1,The Big Carrot Natural Food Market,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",43.678879,-79.297734
2,Beaches Bake Shop,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",43.680363,-79.289692
3,The Beech Tree,"[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",43.680493,-79.288846
4,Grover Pub and Grub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",43.679181,-79.297215
5,Glen Stewart Ravine,"[{'id': '4bf58dd8d48988d162941735', 'name': 'O...",43.676300,-79.294784
6,Upper Beaches,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.680563,-79.292869
7,Glen Stewart Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.675278,-79.294647


In [29]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Beaches Bake Shop,Bakery,43.680363,-79.289692
3,The Beech Tree,Gastropub,43.680493,-79.288846
4,Grover Pub and Grub,Pub,43.679181,-79.297215


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Create a function to repeat the same process for all the Neighborhoods in Toronto.

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
#### Code to run the above function on each neighborhood and create a new dataframe called *roronto_venues*.

In [33]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

Check the size of the resuting dataframe.

In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(1652, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6784,-79.2941,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.6784,-79.2941,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.6784,-79.2941,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,The Beaches,43.6784,-79.2941,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.6784,-79.2941,Grover Pub and Grub,43.679181,-79.297215,Pub


Check how many venues were returned for each neighborhood.

In [35]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,46,46,46,46,46,46
"Brockton, Exhibition Place, Parkdale Village",1,1,1,1,1,1
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",57,57,57,57,57,57
"Cabbagetown, St. James Town",42,42,42,42,42,42
Central Bay Street,97,97,97,97,97,97
"Chinatown, Grange Park, Kensington Market",78,78,78,78,78,78
Christie,13,13,13,13,13,13


#### Number of unique categories can be curated from all the returned venues

In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 206 uniques categories.


## 3. Analyze Each Neighborhood

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


New dataframe size.

In [38]:
toronto_onehot.shape

(1652, 206)

#### Group rows by Neighborhood and by taking the mean of the frequency of occurrence of each category.

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.021739,0.021739,0.000000,0.021739,0.000000,0.000000,...,0.000000,0.00,0.021739,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.00
3,Business Reply Mail Processing Centre 969 Eastern,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.066667,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
6,Central Bay Street,0.000000,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.010309,0.010309,0.00,0.010309,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.025641,0.012821,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.051282,0.000000,0.051282,0.00,0.012821,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
9,Church and Wellesley,0.011494,0.011494,0.011494,0.000000,0.000000,0.011494,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.011494,0.00,0.000000,0.011494,0.00


New dataframe size:

In [40]:
toronto_grouped.shape

(38, 206)

#### Print each neighborhood along with the top 5 most common venues

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.10
1  Clothing Store  0.08
2  Cosmetics Shop  0.04
3      Restaurant  0.03
4           Plaza  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.15
1              Bakery  0.07
2             Theater  0.04
3  Mexican Restaurant  0.04
4       Historic Site  0.02


----Brockton, Exhibition Place, Parkdale Village----
                       venue  freq
0              Women's Store   1.0
1    New American Restaurant   0.0
2                Men's Store   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Coffee Shop  0.13
1         Yoga Studio  0.07
2  Italian Restaurant  0.07
3    Department Store  0.07
4                Park  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  

#### Put that into a *pandas* dataframe

First, write a function to sort the venues in descending order.

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
#neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Clothing Store,Cosmetics Shop,Plaza,Restaurant,Bakery,Theater,Sushi Restaurant,Japanese Restaurant,Tea Room
1,Berczy Park,Coffee Shop,Bakery,Mexican Restaurant,Theater,Historic Site,Farmers Market,Spa,Breakfast Spot,Brewery,Shoe Store
2,"Brockton, Exhibition Place, Parkdale Village",Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Yoga Studio,Deli / Bodega,Restaurant,Sushi Restaurant,Salad Place,Bookstore,Japanese Restaurant,Department Store,Breakfast Spot
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Bar,Café,Restaurant,Gym / Fitness Center,Pub,Park,Bakery,Speakeasy


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [44]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1,
       4, 0, 2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged.head() # check the last columns!
toronto_merged.dropna(inplace=True) 
toronto_merged.head()

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.6784,-79.2941,0.0,Park,Trail,Bakery,Pub,Gastropub,Other Great Outdoors,Health Food Store,Cuban Restaurant,Discount Store,Farmers Market
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538,0.0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Coffee Shop,Grocery Store,Brewery,Bubble Tea Shop,Café,Restaurant,Pub
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.6693,-79.3155,0.0,Sandwich Place,Gym,Ice Cream Shop,Park,Pet Store,Movie Theater,Pub,Liquor Store,Burrito Place,Burger Joint
3,43,M4M,East Toronto,Studio District,43.6561,-79.3406,0.0,Coworking Space,Coffee Shop,Diner,Baseball Field,Garden Center,Performing Arts Venue,Park,Gym,Dog Run,Farmers Market
4,44,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935,4.0,Photography Studio,Park,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Finally, let's visualize the resulting clusters

In [46]:
toronto_merged[20:25]

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.6469,-79.3823,0.0,Coffee Shop,Café,Hotel,Seafood Restaurant,Restaurant,Gastropub,Steakhouse,Bar,Salad Place,Deli / Bodega
21,61,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.6492,-79.3823,0.0,Coffee Shop,Café,Restaurant,Gym,Hotel,Steakhouse,Asian Restaurant,Bar,American Restaurant,Burger Joint
23,64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.6966,-79.412,3.0,Park,Home Service,Trail,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
24,65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.6736,-79.4035,0.0,Sandwich Place,Café,American Restaurant,Pharmacy,Restaurant,Mexican Restaurant,Pub,Middle Eastern Restaurant,Italian Restaurant,Coffee Shop
25,66,M5S,Downtown Toronto,"Harbord, University of Toronto",43.6629,-79.3987,0.0,Café,Bakery,Restaurant,Japanese Restaurant,Bookstore,Italian Restaurant,Sandwich Place,Noodle House,Chinese Restaurant,French Restaurant


In [50]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,-79.2941,0.0,Park,Trail,Bakery,Pub,Gastropub,Other Great Outdoors,Health Food Store,Cuban Restaurant,Discount Store,Farmers Market
1,M4K,-79.3538,0.0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Coffee Shop,Grocery Store,Brewery,Bubble Tea Shop,Café,Restaurant,Pub
2,M4L,-79.3155,0.0,Sandwich Place,Gym,Ice Cream Shop,Park,Pet Store,Movie Theater,Pub,Liquor Store,Burrito Place,Burger Joint
3,M4M,-79.3406,0.0,Coworking Space,Coffee Shop,Diner,Baseball Field,Garden Center,Performing Arts Venue,Park,Gym,Dog Run,Farmers Market
5,M4P,-79.3887,0.0,Dog Run,Food & Drink Shop,Breakfast Spot,Gym,Park,Hotel,Department Store,Doner Restaurant,Fast Food Restaurant,Farmers Market
7,M4S,-79.3853,0.0,Italian Restaurant,Dessert Shop,Café,Sandwich Place,Coffee Shop,Sushi Restaurant,Restaurant,Indian Restaurant,Thai Restaurant,Gas Station
9,M4V,-79.4025,0.0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space
11,M4X,-79.3689,0.0,Coffee Shop,Chinese Restaurant,Café,Bakery,Italian Restaurant,Pizza Place,Restaurant,Sandwich Place,Jewelry Store,Butcher
12,M4Y,-79.383,0.0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Burger Joint,Smoke Shop,Gym,Hotel,Gastropub
13,M5A,-79.3626,0.0,Coffee Shop,Breakfast Spot,Spa,Italian Restaurant,Restaurant,Pub,Mexican Restaurant,Thai Restaurant,Bakery,Beer Store


#### Cluster 2

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,M6K,-79.4198,1.0,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
34,M6P,-79.4198,1.0,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
36,M6S,-79.4198,1.0,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 3

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M4R,-79.4065,2.0,Gym Pool,Playground,Park,Garden,Discount Store,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
8,M4T,-79.3853,2.0,Park,Gym,Grocery Store,Playground,Thai Restaurant,Women's Store,Discount Store,Farmers Market,Event Space,Ethiopian Restaurant
10,M4W,-79.373,2.0,Park,Playground,Grocery Store,Candy Store,Women's Store,Dog Run,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 4

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M5P,-79.412,3.0,Park,Home Service,Trail,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant


#### Cluster 5

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,-79.3935,4.0,Photography Studio,Park,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
